In [1]:
!pip install opencv-python-headless numpy imutils scipy matplotlib

In [3]:
import cv2
import numpy as np
import imutils
from scipy.spatial import distance as dist
from imutils import contours
from imutils import perspective
import matplotlib.pyplot as plt

In [5]:
import cv2
import numpy as np
import imutils
from scipy.spatial import distance as dist
from imutils import contours
from imutils import perspective
import matplotlib.pyplot as plt

# Install any missing libraries
!pip install opencv-python-headless numpy imutils scipy matplotlib

In [7]:
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def calculate_dimensions(image, width):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    edged = cv2.Canny(gray, 50, 100)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    (cnts, _) = contours.sort_contours(cnts)
    pixels_per_metric = None
    output_image = image.copy()

    for c in cnts:
        if cv2.contourArea(c) < 100:
            continue

        box = cv2.minAreaRect(c)
        box = cv2.boxPoints(box) if imutils.is_cv4() else cv2.cv.BoxPoints(box)
        box = np.array(box, dtype="int")
        box = perspective.order_points(box)

        cv2.drawContours(output_image, [box.astype("int")], -1, (0, 255, 0), 2)
        
        for (x, y) in box:
            cv2.circle(output_image, (int(x), int(y)), 5, (0, 0, 255), -1)

        (tl, tr, br, bl) = box
        (tltrX, tltrY) = midpoint(tl, tr)
        (blbrX, blbrY) = midpoint(bl, br)
        (tlblX, tlblY) = midpoint(tl, bl)
        (trbrX, trbrY) = midpoint(tr, br)
        
        dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
        dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
        
        if pixels_per_metric is None:
            pixels_per_metric = dB / width

        dimA = dA / pixels_per_metric
        dimB = dB / pixels_per_metric

        cv2.putText(output_image, "{:.1f}in".format(dimA),
                    (int(tltrX - 10), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,
                    0.65, (255, 255, 255), 2)
        cv2.putText(output_image, "{:.1f}in".format(dimB),
                    (int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
                    0.65, (255, 255, 255), 2)
    
    return output_image

In [ ]:
def capture_image_from_webcam():
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return None

    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Could not read frame.")
            break

        cv2.imshow("Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('c'):
            cv2.imwrite('captured_image.png', frame)
            break

    cap.release()
    cv2.destroyAllWindows()
    return frame

captured_image = capture_image_from_webcam()
if captured_image is not None:
    output_image = calculate_dimensions(captured_image, 0.955)
    plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()